<a href="https://colab.research.google.com/github/Mrrohit0806/Movie-Recommendation-System-Project_02/blob/main/Recommendation_system_source_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import ast
import nltk

movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credit = pd.read_csv('/content/tmdb_5000_credits.csv')

# we merge both data sets for our simple handaling movies.merge()

movies = movies.merge(credit,on = 'title') #  we merge dataset on the basis of title of  movies

# Important colums
# genres , id , keywords, title, overview,cast,crew
movies = movies[['title','id','keywords','genres','overview','cast','crew']]

movies.dropna(inplace = True) # missing data handling

# we have to make colums in right formate

def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

def convert_3(obj):
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter !=3:
       L.append(i['name'])
       counter +=1
    else:
      break
  return L

movies['cast'] = movies['cast'].apply(convert_3)

def fetch_director(obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      L.append(i['name'])
      break
  return L

movies['crew'] = movies['crew'].apply(fetch_director)

movies['overview'] = movies['overview'].apply(lambda x:x.split()) # for convert in list

# remove spaces between names

movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

# create tag colums from concatination of other colums

movies['tags'] = movies['overview'] +  movies['genres'] +  movies['keywords'] +  movies['cast'] +  movies['crew']

new_df = movies[['id','title','tags']]

# add apacing in tag colums

new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

# applying samming on simiral meaning words

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
    return " ".join(y)

new_df['tags'] = new_df['tags'].apply(stem)


#vectorisation of tags

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

vectors = cv.fit_transform(new_df['tags']).toarray()

# measure Cosine distaces between vectors

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

# recomdetion function

def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    # sirting with index position kept
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
      print(new_df.iloc[i[0]].title)
    return

# Wewant to show over movies option on app interface
import pickle

pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

pickle.dump(similarity,open('similarity.pkl','wb'))
